# Compressed Sensing (CS) based ECG compressor

## Table of Contents

TO BE WRITTEN

# Goal of Project

TO BE WRITTEN

Roadmap:
- reproducing idea from paper bla bla bla
- in general: study ("emulate", not really) best solution for a CS-based compressor for ECG to be used with remote-ECG-devices, small, limited storage capability, limited computational power:
    - __phase 1__ compute dictionary (only for adaptive dictionaries) $\Psi$ and measurement matrix $\Phi$ before actually using the device to measure the patient ecg
    - __phase 2__ pass $\Psi$, $\Phi$ to the device, take __already compressed measurements__ $y$ (we'll see that this is core idea of CS)
    - __phase 3__ store only $y$, $\Psi$, $\Phi$ and send them back to _more computationally powerful system_ where recovery happens
- paper focuses also on how such hardware is built, we will be more generic
- exploit data from Physionet.org exactly like the paper did
- test different dictionaries, both _fixed dictionaries_ (_DCT_, _DWT_, _KL_) and _adaptive dictionary learning_ (_MOM_, _K-SVD_)
- test how dimension of measurement matrix $\Phi$ is related to processing speed in __phase 2__
- test different _recovery methods_ ("classic" _l1-minimization_, _LASSO_, _Greedy Algorithms_, _Smooth-L0_, _Baisis Pursuit_), always use newly developed __Kronecker technique__ 
- testing robustness to noise with _additive noise_



# Theoretical Review

## Theory: sparsity and compression

### Understanding Sparsity
__Theoretical Sparsity__

A signal $s \in \mathbb{R}^n$ is considered **$k$-sparse** if it has exactly $k$ non-zero elements, with $k \ll n$. This means that $n-k$ elements of the signal are exactly zero.
$$
s = \begin{pmatrix} s_1 \\ s_2 \\ \vdots \\ s_n \end{pmatrix}
$$
where exactly $k$ elements in $s$ are non-zero, and the remaining $n-k$ elements are zero.


__Practical Sparsity__

In real-world signals, _exact sparsity is rare_. Instead, signals are often _representable_ (see next section) as __approximately sparse__: only $k$ elements _of the sparse representation_ are significant and carry most of the signal's information, the remaining $n-k$ elements have small, negligible values. 

The difference lies in the fact that the $n-k$ coefficients are small but not exactly zero.

### Sparse Representation of Signals

"Most natural signals, such as images and audio, are highly compressible. This compressibility means that, when the signal is written in an appropriate basis, only a few modes are active, thus reducing the number of values that must be stored for an accurate representation. In other words, a compressible signal $x \in \mathbb{R}^n$ may be written as a sparse vector $s \in \mathbb{R}^n$ in a transform basis $\Psi \in \mathbb{C}^{n \times n}$:

$$
x = \Psi s.
$$

If the basis $\Psi$ is generic, such as the Fourier or wavelet basis, then only the few active terms in $s$ are required to reconstruct the original signal $x$, reducing the data required to store or transmit the signal." [2]

### Classic Transformation-Based Compression

A typical transformation-based compression algorithm involves the following steps:

1. __Signal capture__: 
    Fully sense a whole __raw__ signal $x$ and store it. In this project $x$ are the _voltages_ measured by the ECG machine.
2. __Transformation to a sparse domain__:
    The signal $x$ is transformed to a sparse domain, basically we want to find the sparse vector $s \in \mathbb{R}^n$, that contain mostly negligible coefficients.

    We exploit $\Psi \in \mathbb{C}^{n \times n}$ orthogonal basis matrix, also called __dictionary__. Being $\Psi$ an orthonormal basis, it satisfies $\Psi^H \Psi = I$, where $\Psi^H$ is the Hermitian conjugate (conjugate transpose) of $\Psi$, and $I$ is the identity matrix. This implies that $\Psi^{-1} = \Psi^H$, making the transformation and its inverse straightforward.

    Therefore, when $ \Psi $ is an orthonormal basis, applying $ \Psi^H $ to the signal effectively inverts the transformation applied by $ \Psi $, we can use this to obtain sparse representation from original signal:

    $$
    s = \Psi^H x
    $$

    __The use of transforms__:
    On a mathematical note: $\Psi$ is an orthonormal basis composed of functions like Fourier Function, Wavelet, and so on.
    The actual computation of $s$ doesn't actually build a dictionary $\Psi$ to invert and multiplicate to the signal. Instead it directly applies the _transform_ (e.g. FFT, DWT, DCT, ...) to the signal $x$, to immediately obtain _sparse representation_ $s$.

3. __Sparsification__: 
    A fundamental concept is that a threshold is applied to the coefficients, retaining only those that are significant (i.e., above the threshold) and discarding the rest.

    A more detailed view reveals that these steps can be performed using a wide range of techniques, depending on the transform employed, and equivalently, on the choice of dictionary.

    _This will not be explored as it is not the subject of this project, it's a vast and intresting topic, Brunto&Kutz book in reference provide a good reference to explore more..._

4. __Encoding__:
    The retained coefficients and their positions are then encoded for storage or transmission. 

    _Another huge chapter that will not be explored here, again you can refer to the referenced book for more_

__Complexity__

Such methods can be _extremely_ effective, but they require a _thresholding/sparsification_ step, which introduces non-linearity and computational complexity. 

In the following is shown that CS-based methods can provide an alternative solution with different advantages...

<center>
    <img src="./.img/MethodsComparison.png" alt="MethodsComparison.png" width="600">
</center>

### Compressed Sensing (CS)

"Mathematically, compressed sensing exploits the _sparsity of a signal_ in a __generic basis__ to achieve full signal reconstruction from surprisingly few measurements.

If a __signal $x$ is k-sparse in $\Psi$ (it's a requirement),__ then instead of measuring $x$ directly (n measurements) and then compressing, it is possible to collect dramatically fewer randomly chosen or compressed measurements and then solve for the non-zero elements of s in the transformed coordinate system." [2]


#### Measurement

Instead of acquiring all $n$ samples, a reduced set of $m$ measurements is obtained directly by projecting the signal $x$  onto a measurement matrix $\Phi$, storing a _compressed measurement_ $y$:

$$
y = \Phi x
$$


where:
- $x \in \mathbb{R}^n$ _real_ signal coming from sensors
- $y \in \mathbb{R}^m$ _compressed measurement_
- $\Phi \in \mathbb{R}^{m \times n}$ with $m \ll n$ is the _measurement matrix_.

__Key concept__:
In the measurement phase the _sparse representation_ $s$ is __not__ computed, we directly apply the _measurement matrix_ to the _real_ signal $x$. 

$\Phi$ does not simply "select" $m$ out of $n$ coefficients out of $x$. Instead, $\Phi$ typically contains random or structured elements that ensure the measurements $y$ retain sufficient information to __later recover__ the sparse signal $s$. 

_Next chapter will explore measurement matrix topic_


#### Recovery

With knowledge of $s \in \mathbb{R}^n$ _sparse representation_ of $x$ through $\Psi$ _dictionary_, it is possible to recovery $x$ itself as previously shown with:
$$
x = \Psi s
$$

Thus the goal of compressed sensing is to find the __sparsest__ vector $s$ that is consistent with:

$$
y = \Phi x = \Phi \Psi s
$$

where (again):
- $x \in \mathbb{R}^n$ _real_ signal coming from sensors
- $y \in \mathbb{R}^m$ _compressed measurement_
- $\Psi \in \mathbb{R}^{n \times n}$ is the _dictionary_ (same as explained in previous section)
- $\Phi \in \mathbb{R}^{m \times n}$ with $m \ll n$ is the _measurement matrix_.
- $s \in \mathbb{R}^n$ is the _sparse representation_ of $x$ in $\Psi$

__Non convex problem__

"Such system of equations is __under-determined__ since there are infinitely many consistent solution $s$. The __sparsest solution__ is the one that satisfies:

$$
\hat{s} = \arg_{s} \min \|s\|_0 \text{ subject to } y = \Phi \Psi \alpha
$$

where $\min \|s\|_0$ denotes the $\ell_0$-pseudo-norm, given by the _non-zero entries_, also referred as the _cardinality_ of $s$.

The optimization is non-convex, and in general, the solution can only be found with a brute-force search that is combinatorial in $n$ and $K$. In particular, all possible $K$-sparse vectors in $\mathbb{R}^n$ must be checked; if the exact level of sparsity $K$ is unknown, the search is even broader. Because this search is combinatorial, solving such minimization is intractable for even moderately large $n$ and $K$, and the prospect of solving larger problems does not improve with Moore’s law of exponentially increasing computational power."[2]

__Convex equivalent problem__

Fortunately, under certain conditions on the measurement matrix $\Phi$, it is possible to relax the optimization to a convex $\ell_1$-minimization.

$$
\hat{s} = \arg_{s} \min \|s\|_1 \text{ subject to } y = \Phi \Psi \alpha
$$

__In the presence of noise__, the recovery problem is modified to:

$$
\hat{s} = \arg_{s} \min \|s\|_1 \text{ subject to } \|y - \Phi \Psi s\|_2 \leq \epsilon
$$

where $\epsilon$ is a bound on the noise level.

There are very specific conditions that must be met for the $\ell_1$-minimization to converge with high probability to the sparsest solution of $\ell_0$-minimization. They can be summarized as follows:
- __Incoherence__: 
    A critical concept in compressed sensing is the _incoherence_ between the measurement matrix $\Phi$ and the dictionary $\Psi$. Incoherence refers to the property that ensures that the rows of $\Phi$ are not too similar to the columns of $\Psi$. This incoherence is vital because it allows the sparse information in the signal $x$ (which is represented in the domain of $\Psi$) to be evenly spread across the measurements $y$. This spreading ensures that no single measurement in $y$ captures too much or too little information about the signal $x$, which is essential for accurate recovery of the sparse signal $s$ from the measurements $y$.

- __Recoverability Condition:__ 
    A __$K$-sparse__ signal $s \in \mathbb{R}^n$ can be properly recovered after Compressive Sensing (CS) if the number of measurements $m$ satisfies:

    $$
    m \geq C K \log\left(\frac{n}{K}\right)
    $$

    where $C$ is a constant that depends on how __incoherent__ $\Phi$ and $\Psi$ are. This condition ensures that enough measurements are taken to accurately recover the sparse signal, accounting for both sparsity and the ambient dimension $n$.

    The recoverability condition is a practical guideline that tells you how many measurements $m$ you need to take to ensure that a $k$-sparse signal $s \in \mathbb{R}^n$ can be recovered accurately. The $\log\left(\frac{n}{k}\right)$ term accounts for the dimensionality reduction that occurs when mapping an $n$-dimensional signal into an $m$-dimensional measurement space.

"Roughly speaking, these two conditions guarantee that the matrix $\Phi |Psi$ acts as a unitary transformation on K-sparse vectors $s$, preserving relative distances between vectors and enabling almost certain signal reconstruction with $\ell_1$ convex minimization. This is formulated precisely in terms of the restricted isometry property (RIP) that follows."[2]

__Restricted Isometry Property (RIP):__
"The RIP is a property of the matrix $A = \Phi \Psi$ that provides a condition under which the matrix will behave well with respect to sparse signals. Specifically, for a matrix $A$ to satisfy the RIP of order $k$ with a constant $\delta_k$, it must hold that:

$$
(1 - \delta_k) \|x\|_2^2 \leq \|A x\|_2^2 \leq (1 + \delta_k) \|x\|_2^2
$$

for all $k$-sparse vectors $x$. Here, $\delta_k$ is the smallest constant such that this inequality holds, and it should be close to zero. This ensures that the matrix $A$ approximately preserves the Euclidean length (and hence the geometry) of all $k$-sparse signals, meaning the measurements are nearly isometric.


__LASSO Compressed Sensing__

Alternatively, the signal can be recovered using the LASSO formulation, which balances the $\ell_1$ norm of the signal with the fidelity to the measurements:

$$
\hat{s} = \arg \min \left(\frac{1}{2} \|y - \Phi \Psi s\|_2^2 + \lambda \|s\|_1 \right)
$$

Here, $\lambda$ is a regularization parameter that controls the trade-off between the sparsity of the solution and the accuracy of the reconstruction. A larger $\lambda$ emphasizes sparsity, while a smaller $\lambda$ emphasizes data fidelity.

Note that $LASSO$ formulation already accounts for noise using the $\ell_2$ minimization on $\|y - \Phi \Psi s\|_2^2$, instead of enforcing the condition $y = \Phi \Psi s$.

### References for this chapter 
(some parts are directly quoted from source)

[1] __Section 2 "Compressed Sensing" of the Paper__:
   Izadi, V., Shahri, P.K., & Ahani, H. (2020). A compressed-sensing-based compressor for ECG. *Biomedical Engineering Letters*, 10, 299–307. https://doi.org/10.1007/s13534-020-00148-7

[2] __Chapter 3.1 "Sparsity and Compressed Sensing" of the Book__:
   Brunton, S. L., & Kutz, J. N. (2022). *Data-Driven Science and Engineering: Machine Learning, Dynamical Systems, and Control* (2nd ed.). Cambridge University Press.


## Theory: metrics and choices

TO BE WRITTEN

Roadmap:
- also here choices strongly based on paper bla bla bla
- _Compression rate (CR)_ of 75%, obtained directly "by construction" in the process (through _measurement matrix_)
- _complexity of algorithm in recording_ directly bind to _power consumption_, very relevant for remote-ecg-recorders
- _emergency ecg_ , such in ambulatory require _processing speed_, both during sampling and in reconstruction
    - both previous point are related to _optimal choice of measurement matrix_, as it will be explained later
- _accuracy_ through quantitive metrics: _PRD_, _SNR_
    - especially important to evaulate different dictionaries adn different methods of reconstruction

## Theory: measurement matrix

TO BE WRITTEN

Roadmap:
- Relation between $\Phi$ dimension and speed of "sensing phase" (phase 2)
- Relation between randomicity and RIP property

## Theory: dictionaries

### _fixed dictionaries_ vs _adaptive dictionary learning_
TO BE WRITTEN

---

### Fixed dictionaries

#### DCT
TO BE WRITTEN

#### DWT
TO BE WRITTEN

#### KL
TO BE WRITTEN

---

### Adaptive Dictionary Learning
TO BE WRITTEN

#### MOM
TO BE WRITTEN

#### K-SVD
TO BE WRITTEN

## Theory: reconstruction of the signal

### Reconstruction methods

(we already explained the maths before, here is practival parts)

#### "classic" _l1-minimization_ problem
TO BE WRITTEN

#### LASSO
TO BE WRITTEN

#### Greedy Algorithms
TO BE WRITTEN

#### Smooth-L0
TO BE WRITTEN

#### Basis Pursuit
TO BE WRITTEN

---

### Kronecker Techinque

TO BE WRITTEN

# Code implementation (python)


---
__Pre-sampling phase__

- Generate dictionary $\Psi$
- Generate Measurement Matrix $\Phi$

---
__Sampling phase__
- $y = \Phi \Psi s$, simulate what would happen on device (_compute block by block, later concatenate results_)

---
__Recovery phase__
- Reconstruct signal

---
__Evaluate result__
1. Sampling faster for smaller $\Phi$?
2. Which dictionary are the best?
3. Which recovery method is best?
4. Robustness to noise?

---

## Code: dictionary

- First step of __Pre-sampling phase__
- Develop code that outputs a dictionary $\Psi \in \mathbb{R}^{d \times d}$
- $d$ will be the block dimension. 
- Code will be able to create the dictionary with one of the methods
    - Fixed dictionaries: DCT, DWT, KL
    - Adaptive Dictionary Learning: MOM, K-SVD



In [ ]:
import numpy as np
import pywt

def create_wavelet_basis(N, wavelet_name='db4'):
    """
    Create the wavelet basis matrix for a given signal dimension and wavelet type.

    Parameters:
    N (int): The dimension of the signal.
    wavelet_name (str): The name of the wavelet (default is 'db4').

    Returns:
    waveletBasis (ndarray): The combined basis matrix, where the first N/2 rows are the
                            approximation basis and the second N/2 rows are the detail basis.
    """
    # Load the specified wavelet
    wavelet = pywt.Wavelet(wavelet_name)
    
    # Extract the scaling (low-pass) and wavelet (high-pass) filter coefficients
    h = wavelet.dec_lo
    g = wavelet.dec_hi
    
    # Length of the filter
    L = len(h)
    
    # Initialize matrices to store the basis vectors
    Phi = np.zeros((N//2, N))
    Psi = np.zeros((N//2, N))
    
    # Construct the approximation (Phi) and detail (Psi) basis matrices
    for i in range(N//2):
        Phi[i, i*2:i*2+L] = h
        Psi[i, i*2:i*2+L] = g
    
    # Combine Phi and Psi into a single matrix
    waveletBasis = np.vstack((Phi, Psi))
    
    return waveletBasis




## Code: measurement matrix

- Second step of __Pre-sampling phase__
- Simple code to produce a $\Phi \in \mathbb{R}^{d/4 \times d}$ measurement matrix
- Coefficient of $\Phi$ will be random (_Bernoulli_) extraction of $-1,1$

## Code: sampling phase

- Only step of __Sampling phase__ 
- Compute $y_{j} = \Phi \Psi s_{j}$ on each $j$-th block of signal of dimension $d$
- Progressively attach new $y_{j}$ to previous unitil you obtain $y$ _compressed measurement_ (of whole signal $s$)

## Code: recovery

- Only step of __Recovery phase__
- Code that will be able to use _one of the selected methods_ to reconstruct an apporoximation of original signal $s$ from the _compressed measurement_ $y$
    - $l1$-minimization
    - LASSO
    - Greedy
    - Smooth-L0
    - Basis Pursuit
- __All methods (if possible) must exploit Kronecker Technique__

## Data: MIT–BIH Arrhythmia Database

USE THE ONE IN THE PREVIOUS NOTEBOOK

## Code: test for best dictionary

- Test all dictionaries to show with big number of data which are the one to perform better (we expect DWT to be the best of fixed, and that adaptive are better than fixed in general)
- __#Records:__ Test on __MULTIPLE patients__ records is a MUST, especially to show that adaptive are better 
- __#Dictionaries:__ Test all dictionaries, that's what we are doing ...
- __#ReconstructionMethods:__ Test with __only one reconstruction method__

## Code: test for best reconstruction method

- Test to find which recontruction method is the best
- __#Records:__ Test on a __single patient__ record should be fine 
- __#Dictionaries:__ Test with __a single dictionary type__
- __#ReconstructionMethods:__ Test with __all reconstruction methods__

## Code: test for correct block dimension

- Test to show that __sampling phase process speed is inversly proportional to block dimension__
- __#Records:__ Test on a __single patient__ record should be fine 
- __#Dictionaries:__ Test with __a single dictionary type__ (USE BEST!)
- __#ReconstructionMethods:__ Test with __only one reconstruction method__ (USE BEST!)

## Code: test for noise robustness

- Test if robust to noise by adding noise to the signal
- __#Records:__ Test on a __single patient__ record should be fine 
- __#Dictionaries:__ Test with __a single dictionary type__ (USE BEST!)
- __#ReconstructionMethods:__ Test with __only one reconstruction method__ (USE BEST!)

# Conclusions

TO BE WRITTEN

# References

Sources are cited also at the end of each chapter, with a number (e.g. [2]) in the text for directly quoted parts.

TO BE WRITTEN

# Appendix

TO BE WRITTEN?

For instance explanation of what ecg is